In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 4.2 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
     |████████████████████████████████| 6.6 MB 66.0 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW
import time
from tqdm import tqdm
import string
from scipy.special import softmax

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Multilingual Classification/train.xlsx'
test_path = '/content/drive/MyDrive/Colab Notebooks/Multilingual Classification/test.xlsx'
valid_path = '/content/drive/MyDrive/Colab Notebooks/Multilingual Classification/valid.xlsx'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
def source_preprocess(text):
  text = text.lower()
  text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
  return text

def target_preprocess(text):
  punctuations = string.punctuation + "؟«»؛،"
  text = text.translate(str.maketrans(punctuations, ' ' * len(punctuations)))
  return text

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, dataframe, labels, tokenizer, features, features_preprocess, max_len):
        self.dataframe = dataframe
        self.labels = labels
        self.tokenizer = tokenizer
        self.features_preprocess = features_preprocess
        self.max_len = max_len
        self.features = features
        
    def get_dataset(self):
      input_ids = []
      attention_masks = []
      targets = []
      for index, row in self.dataframe.iterrows():
        if row['category'] not in self.labels.keys():
          continue
        target = self.labels[row['category']]
        combined_encoded = []
        for i, (feature, preprocess) in enumerate(zip(self.features, self.features_preprocess)):
          combined_encoded += self.tokenizer.encode(preprocess(row[feature]), add_special_tokens=False)
          if i < len(self.features) - 1:
            combined_encoded += [self.tokenizer.sep_token_id]
        if len(combined_encoded) > self.max_len:
          combined_encoded = combined_encoded[:self.max_len]
        attention_mask = [1] * len(combined_encoded)
        padded_combined_encoded = combined_encoded + [1] * (self.max_len - len(combined_encoded))
        padded_attention_mask = attention_mask + [0] * (self.max_len - len(attention_mask))
        input_ids.append(padded_combined_encoded)
        attention_masks.append(padded_attention_mask)
        targets.append(target)
      input_ids = torch.tensor(input_ids)
      attention_masks = torch.tensor(attention_masks)
      targets = torch.tensor(targets)
      return TensorDataset(input_ids, attention_masks, targets)

def create_data_loader(dataframe, labels, tokenizer, features, features_preprocess, max_len, batch_size):
    dataset = CustomDataset(dataframe, labels, tokenizer, features, features_preprocess, max_len)
    return DataLoader(dataset.get_dataset(), batch_size = batch_size)

In [ ]:
def eval(model, dataloader, data_len, device):
  model.eval()
  y_preds = np.array([])
  y_probs = None
  y_trues = np.array([])
  losses = []
  correct_preds = 0
  with torch.no_grad():
    for d in tqdm(dataloader):
      input_ids = d[0].to(device)
      attention_mask = d[1].to(device)
      targets = d[2].to(device)
      y_trues = np.concatenate((y_trues, targets.cpu().numpy().copy()), axis=0)
      outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
      y_prob = softmax(outputs.logits.cpu().tolist(), axis=1)
      if y_probs is None:
        y_probs = y_prob
      else:  
        y_probs = np.concatenate((y_probs, y_prob), axis=0)
      _, preds = torch.max(outputs.logits, dim=1)
      y_preds = np.concatenate((y_preds, preds.cpu().numpy().copy()), axis=0)
      loss = outputs.loss
      correct_preds += torch.sum(preds == targets)
      losses.append(loss.item())
  acc = correct_preds / data_len
  loss = np.mean(losses)
  return y_probs, y_preds, y_trues, acc, loss

In [ ]:
def train(model, train_dataloader, eval_dataloader, train_data_len, eval_data_len, device, epochs):
  optimizer = AdamW(model.parameters(), lr=3e-5)
  for epoch in range(epochs):
    start = time.time()
    model.train()
    losses = []
    correct_preds = 0
    for d in tqdm(train_dataloader):
      input_ids = d[0].to(device)
      attention_mask = d[1].to(device)
      targets = d[2].to(device)
      outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
      _, preds = torch.max(outputs.logits, dim=1)
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      correct_preds += torch.sum(preds == targets)
      losses.append(loss.item())
    train_acc = correct_preds / train_data_len
    train_loss = np.mean(losses)
    _, _, _, eval_acc, eval_loss = eval(model, eval_dataloader, eval_data_len, device)
    iter_time = time.time() - start
    print(f"epoch {epoch + 1} -- train accuracy: {train_acc}, train loss: {train_loss}, validation accuracy: {eval_acc}, validation loss: {eval_loss}, epoch time: {int(iter_time)} (s)")

In [ ]:
train_df = pd.read_excel(train_path)
test_df = pd.read_excel(test_path)
valid_df = pd.read_excel(valid_path)
labels = {"quran": 0, "bible": 1, "mizan": 2}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
train_loader = create_data_loader(train_df, labels, tokenizer, ['source'], [source_preprocess], 128, 32)
eval_loader = create_data_loader(valid_df, labels, tokenizer, ['targets'], [target_preprocess], 128, 32)
test_loader = create_data_loader(test_df, labels, tokenizer, ['targets'], [target_preprocess], 128, 32)
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)
model.to(device)
train(model, train_loader, eval_loader, len(train_df), len(valid_df), device, 10)
y_probs, y_preds, y_trues, test_acc, test_loss = eval(model, test_loader, len(test_df), device)
print(f"test accuracy: {test_acc}, test loss: {test_loss}")
print()
print(classification_report(y_trues.tolist(), y_preds.tolist(), labels=list(labels.values()), target_names=list(labels.keys())))
y_trues_onehot = []
for y_true in y_trues.tolist():
  temp = [0] * len(labels.keys())
  temp[int(y_true)] = 1
  y_trues_onehot.append(temp)
print(roc_auc_score(y_trues_onehot, y_probs.tolist()))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

epoch 1 -- train accuracy: 0.8319841027259827, train loss: 0.42535914353988497, validation accuracy: 0.7314814925193787, validation loss: 0.7875272964729982, epoch time: 304 (s)


100%|██████████| 85/85 [00:18<00:00,  4.53it/s]


epoch 2 -- train accuracy: 0.9385713934898376, train loss: 0.17620564140332154, validation accuracy: 0.7818518280982971, validation loss: 0.5977879853809581, epoch time: 303 (s)


100%|██████████| 85/85 [00:18<00:00,  4.59it/s]


...

epoch 9 -- train accuracy: 0.9898412227630615, train loss: 0.031091301742516255, validation accuracy: 0.7807407379150391, validation loss: 0.9081368716324076, epoch time: 303 (s)


100%|██████████| 85/85 [00:18<00:00,  4.60it/s]


epoch 10 -- train accuracy: 0.992618978023529, train loss: 0.023373370606119527, validation accuracy: 0.7559259533882141, validation loss: 1.0853802873807794, epoch time: 303 (s)


100%|██████████| 85/85 [00:18<00:00,  4.59it/s]

test accuracy: 0.7451851963996887, test loss: 1.119846287545036

              precision    recall  f1-score   support

       quran       0.74      0.72      0.73       900
       bible       0.84      0.55      0.67       900
       mizan       0.70      0.97      0.81       900

    accuracy                           0.75      2700
   macro avg       0.76      0.75      0.74      2700
weighted avg       0.76      0.75      0.74      2700

0.9211016460905349
